# 

# Week 4: Regularization and Hyperparameter Tuning

This week, we take a dive into how to improve the performance of our machine learning models. We will focus on regression problems, where we try to predict a numerical value, but the ideas herein are pretty universal across supervised machine learning.

First, **regularization** methods sometimes help improve a model's ability to make good predictions on the test set, often at the expense of training accuracy. We focus on some methods developed by mathematician Andrey Tikhonov and used for solving ill-posed inverse problems. Some special cases of his methods and new innovations have become incredibly popular in machine learning.

Second, machine learning models we have seen have trainable parameters determined by a learning algorithm, such as the coefficients in linear regression and the shape and prototype parameters in radial basis functions. **Hyperparameters** are numbers we set or decisions we make before running learning algorithms. Wednesday, we focus on effectively making these choices for performance, or **tuning** them.

## Lecture 12: Regularization and Overfitting

The problem of **overfitting** is an issue where a machine learning model fits too strongly to the training data, which reduces its ability to generalize to make good predictions on the test set. High performance on the test set is typically our most important goal, because this measures how the model performs on data it has not seen before, indicating the model should perform well on real-world data, assuming the test data are representative of the data we hope to predict.

In [22]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import scale

In [23]:
3+3

6

In [1]:
class RegularizedLinearRegression:
    
    def __init__(self, alpha = 0.001, lambda1 = 0, lambda2 = 0):
        # save variables to object memory
        self.alpha = alpha
        #self.lambda1 = lambda1
        self.lambda2 = lambda2
        
    # fit the model to the data
    def fit(self, X, y, epochs = 1000, update = 100):
        # find the dimension of the data
        d = X.shape[1]
        
        # initialize the model parameters
        self.theta0 = np.random.uniform()
        self.theta = np.random.uniform(size = d)
        
        # compute the means of X and y
        yMean = np.mean(y)
        xMean = np.mean(X, axis = 0)
        
        # standardize the data
        X = scale(X)
        y = scale(y)
        
        # compute a scaling parameter
        eta = np.linalg.norm(X)
        
        # train the model
        for i in range(epochs):
            # compute the predicted y values
            predictions = self.theta0 + X @ self.theta
            
            # compute the error
            error = predictions - y
            
            # compute the sum of squared errors
            sse = np.sum(error ** 2)
            
            # compute the loss
            loss = sse + self.lambda2 * np.linalg.norm(self.theta) ** 2 #+ self.lambda1 * np.sum(np.abs(self.theta))

            # print an update
            if i % update == 0:
                print('Epoch', i, '\tLoss', loss)
                                
            # weight update for the bias
            self.theta -= self.alpha * (X.T @ error + 2 * self.lambda2 * self.theta)
            self.theta0 = yMean - xMean @ self.theta

            # weight update for theta (where soft-threholding follows)
            #print(error)
            #print(X.T @ error)
            #print(np.sign(self.theta))
            #print(eta * (X.T @ error + 2 * self.lambda2 * self.theta + self.lambda1 * np.sign(self.theta)))
            #self.theta -= eta * (X.T @ error + 2 * self.lambda2 * self.theta + self.lambda1 * np.sign(self.theta))
            
            # soft-threholding for the LASSO            
            #self.theta = np.sign(self.theta) * np.maximum(np.zeros([1, d]), np.abs(self.theta) - self.alpha * eta)

    # predict the output from input (testing) data
    def predict(self, X):
        # return the predicted outputs
        return self.theta0 + X @ self.theta
    
    # soft-threshold function for LASSO with normalized data
    def soft_threshold(rho, lamda):
        if rho < -lamda:
            return (rho + lamda)
        elif rho > lamda:
            return (rho - lamda)
        else: 
            return 0
        
    def coordinate_descent_lasso(self):

SyntaxError: unexpected EOF while parsing (<ipython-input-1-d399e1ee357c>, line 75)

In [2]:
X = np.array([[6], [7], [8], [9], [10]])
y = np.array([1, 2, 3, 3, 4])

model = RegularizedLinearRegression(lambda2 = 0.1)
model.fit(X, y)
predictions = model.predict(X)

# print the predictions
print('The predicted y values are', predictions)

# print the real y values
print('The real y values are', y)

# print the beta values
parameters = np.concatenate((np.atleast_1d(model.theta0), model.theta))
print('The theta values are', parameters)

# plot the training points
plt.scatter(X, y, label = 'Training Data')

# plot the fitted model with the training data
xModel = np.linspace(6,10,100)
yModel = parameters[0] + parameters[1]*xModel
lineFormula = 'y={:.3f}+{:.3f}x'.format(parameters[0], parameters[1])
plt.plot(xModel, yModel, 'r', label = lineFormula)

# add a legend
plt.legend()

# return quality metrics
print('The r^2 score is', r2_score(y, predictions))
print('The mean squared error is', mean_squared_error(y, predictions))
print('The mean absolute error is', mean_absolute_error(y, predictions))

NameError: name 'np' is not defined

## Lecture 12: Hyperparameter Tuning and Crossvalidation